In [1]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive_output, FloatSlider, HBox, VBox, Select, Button

# Initial triangle coordinates
triangle = np.array([[0, 0], [1, 0], [0.5, 1]])
# Initial triangle coordinates
triangle = np.array([[0, 0], [1, 0], [0.5, 1]])

def plot_shape(ax, shape, *args, **kwargs):
    shape_closed = np.concatenate((shape, shape[:1]), axis=0)
    ax.plot(shape_closed[:, 0], shape_closed[:, 1], *args, **kwargs)


def plot_matrix(ax, mat, color_array):
    ax.axis('tight')
    ax.axis('off')
    formatted_mat = [[f'{value:.2f}' for value in row] for row in mat]  # Format matrix values to 2 decimal places
    tb = ax.table(cellText=formatted_mat, cellColours=color_array,
                  loc='center', cellLoc='center', rowLabels=[''] * 3)
    tb.auto_set_font_size(False)
    tb.set_fontsize(18)
    tb.scale(1.5, 2)

def update(shape_type='Triangle', scale=1.0, rotation_angle=0.0, trans_x=0.0, trans_y=0.0):
    fig, axs = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={'width_ratios': [3, 1]})
    
    ax = axs[0]

    ax.grid(True)  # Display grid
    ax.set_aspect('equal')
    ax.set_xlim([-2, 4])
    ax.set_ylim([-2, 3])

    # Create transformation matrix
    rotation_matrix = np.array([[np.cos(np.radians(rotation_angle)), -np.sin(np.radians(rotation_angle)), trans_x],
                                [np.sin(np.radians(rotation_angle)), np.cos(np.radians(rotation_angle)), trans_y],
                                [0, 0, 1]])
    scaling_matrix = np.array([[scale, 0, 0],
                               [0, scale, 0],
                               [0, 0, 1]])
    transformation_matrix = np.dot(rotation_matrix, scaling_matrix)

    # Determine which cells to color based on the last slider moved
    color_array = np.full((3, 3, 3), 1.0)  # default color
    color_array[2, 0] = color_array[2, 1] = color_array[2, 2] = [0.8, 0.8, 0.8] 
    if last_moved_slider == "Angle":
        color_array[0:2, 0:2] = [0.8, 1, 0.8]  # green for rotation
    elif last_moved_slider == "Translation X":
        color_array[0, 2] = [0.8, 1, 0.8]  # green for translation in X
    elif last_moved_slider == "Translation Y":
        color_array[1, 2] = [0.8, 1, 0.8]  # green for translation in Y
    elif last_moved_slider == "Scale":
        color_array[0, 0] = color_array[1, 1] = [0.8, 1, 0.8]  # green for scaling


    # Determine which shape to plot based on the selection
    if shape_type == 'Triangle':
        # Plot original triangle
        plot_shape(ax, triangle, 'k--')  # Plotting in black dashed line
        # Convert triangle to homogeneous coordinates
        homog_triangle = np.concatenate((triangle, np.ones((3, 1))), axis=1)
        # Apply transformation matrix
        transformed_triangle_homog = np.dot(homog_triangle, transformation_matrix.T)
        # Convert back to Cartesian coordinates
        transformed_triangle = transformed_triangle_homog[:, :2]
        # Plot transformed shape
        plot_shape(ax, transformed_triangle, 'r-')
    elif shape_type == 'Coordinate System':
        # Plot original coordinate system
        ax.plot([0, 1], [0, 0], 'k--')  # x-axis in black dashed line
        ax.plot([0, 0], [0, 1], 'k--')  # y-axis in black dashed line
        # Define original coordinate system
        origin = np.array([[0, 0, 1]])  # origin point in homogeneous coordinates
        x_axis = np.array([[1, 0, 1]])  # x-axis endpoint in homogeneous coordinates
        y_axis = np.array([[0, 1, 1]])  # y-axis endpoint in homogeneous coordinates

        # Plot original coordinate system
        ax.plot([origin[0, 0], x_axis[0, 0]], [origin[0, 1], x_axis[0, 1]], 'b-')  # x-axis in blue
        ax.plot([origin[0, 0], y_axis[0, 0]], [origin[0, 1], y_axis[0, 1]], 'g-')  # y-axis in green

        # Apply transformation matrix to coordinate system
        transformed_origin = np.dot(origin, transformation_matrix.T)
        transformed_x_axis = np.dot(x_axis, transformation_matrix.T)
        transformed_y_axis = np.dot(y_axis, transformation_matrix.T)

        # Plot transformed coordinate system
        ax.plot([transformed_origin[0, 0], transformed_x_axis[0, 0]],
                [transformed_origin[0, 1], transformed_x_axis[0, 1]], 'b-')  # transformed x-axis in blue
        ax.plot([transformed_origin[0, 0], transformed_y_axis[0, 0]],
                [transformed_origin[0, 1], transformed_y_axis[0, 1]], 'g-')  # transformed y-axis in green

    # Display transformation matrix next to the shape
    ax_matrix = axs[1]
    plot_matrix(ax_matrix, transformation_matrix, color_array)  # passing color_array argument

    plt.show()

# Reset function
def reset_values(button):
    global last_moved_slider
    scale_slider.value = 1.0
    rotation_slider.value = 0.0
    trans_x_slider.value = 0.0
    trans_y_slider.value = 0.0
    shape_select.value = 'Triangle'
    last_moved_slider = None
    #update()


# Create sliders for interactive visualization
scale_slider = FloatSlider(min=0.1, max=2.0, value=1.0, description='Scale')
rotation_slider = FloatSlider(min=-180, max=180, value=0, description='Angle')
trans_x_slider = FloatSlider(min=-2, max=2, value=0, description='Translation X')
trans_y_slider = FloatSlider(min=-2, max=2, value=0, description='Translation Y')



last_moved_slider = None  # Global variable to store the name of the last moved slider

def on_slider_change(change):
    global last_moved_slider
    # Update the global variable with the name of the moved slider
    last_moved_slider = change['owner'].description
    # Call the update function
    #update()

# Attach the on_slider_change function to each slider
scale_slider.observe(on_slider_change, names='value')
rotation_slider.observe(on_slider_change, names='value')
trans_x_slider.observe(on_slider_change, names='value')
trans_y_slider.observe(on_slider_change, names='value')


# Create selection widget for shape type
shape_select = Select(
    options=['Triangle', 'Coordinate System'],
    value='Triangle',
    description='Shape Type:',
    disabled=False
)

# Create a reset button
reset_button = Button(description="Reset")
reset_button.on_click(reset_values)



# Create the interactive output object
interactive_plot = interactive_output(update, {'shape_type': shape_select, 'scale': scale_slider, 'rotation_angle': rotation_slider,
                                               'trans_x': trans_x_slider, 'trans_y': trans_y_slider})

# Display the selection widget, sliders, and the interactive plot
display(HBox([VBox([ shape_select, scale_slider, rotation_slider, trans_x_slider, trans_y_slider,reset_button]), interactive_plot]))
